In [31]:
# add this import for running in jupyter notebook
from kinconnect_api.config import MONGO_CONNECTION_STRING

from pymongo import MongoClient
# Connect to MongoDB
client = MongoClient(MONGO_CONNECTION_STRING)
db = client['kinconnect']
profiles_collection = db['profiles']


# Find all profiles in the profiles collection
profiles = list(profiles_collection.find({}))

from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain.schema import Document
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_fireworks import FireworksEmbeddings
from dotenv import load_dotenv

load_dotenv()

def get_chunks(profile):
    headers_to_split_on = [('##', "question")]
    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on
    )

    splits = markdown_splitter.split_text(profile['form_submission'])
    for idx, split in enumerate(splits):
        split.metadata['profile_id'] = str(profile['_id'])
        split.metadata['name'] = profile['name']
        # print(f"Metadata: {profile}")

    return splits


all_chunks = [chunk for profile in profiles for chunk in get_chunks(profile)]

# !! ONLY RUN THIS ONCE
# vector_store = MongoDBAtlasVectorSearch.from_documents(
#     documents = all_chunks,
#     embedding = FireworksEmbeddings(model="nomic-ai/nomic-embed-text-v1.5"),
#     collection = db['profile_chunks'],
#     index_name = "profile_chunks"
# )


In [33]:

all_chunks = [chunk for profile in profiles for chunk in get_chunks(profile)]
vector_store2 = MongoDBAtlasVectorSearch.from_connection_string(
    connection_string = MONGO_CONNECTION_STRING,
    namespace = "kinconnect.profile_chunks",
    embedding = FireworksEmbeddings(model="nomic-ai/nomic-embed-text-v1.5"),
    index_name = "profile_chunks"
)

In [38]:
import pandas as pd
def get_match_profiles(query_text):
    docs = vector_store1.similarity_search_with_score(query_text, k=5)
    profile_names = [doc.metadata['name'] for doc, score in docs]
    profiles = list(profiles_collection.find({'name': {'$in': profile_names}}))
    return profiles

profile_matches = get_match_profiles("A profile of a person that worked at Google")
df = pd.DataFrame(profile_matches)
df.drop(columns=['_id', 'form_submission'], inplace=True)
df


KeyError: "['_id', 'form_submission'] not found in axis"